In [37]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

# Read data from DataSets
mean0 = pd.read_csv("data/DS1_m_0.txt", header=None)
mean1 = pd.read_csv("data/DS1_m_1.txt", header=None)
cov = pd.read_csv("data/DS1_cov.txt", header=None)
# Drop end column (not useful to us)
mean0.drop([20], axis=1, inplace=True)
mean1.drop([20], axis=1, inplace=True)
cov.drop([20], axis=1, inplace=True)
# Change to matrices
mean0_m = mean0.as_matrix()[0]
mean1_m = mean1.as_matrix()[0]
cov_m = cov.as_matrix()[0]

# Generate 2000 examples for each class
dataEx = 2000
class0 = pd.DataFrame(np.random.multivariate_normal(mean0_m, cov, dataEx))
class1 = pd.DataFrame(np.random.multivariate_normal(mean1_m, cov, dataEx))
# Add classification column
class0[20] = 0
class1[20] = 1
class0_m = class0.as_matrix()
class1_m = class1.as_matrix()

# Split data into train, valid, test sets
allData = np.concatenate((class0_m, class1_m), axis=0)
np.random.shuffle(allData)
trainSet = allData[0:2400]
validSet = allData[2400:3200]
testSet = allData[3200:4000]

# Save completed datasets
pd.DataFrame(allData).to_csv("dataGenerated/DS1_data.csv", index=False, header=False)
pd.DataFrame(trainSet).to_csv("dataGenerated/DS1_train.csv", index=False, header=False)
pd.DataFrame(validSet).to_csv("dataGenerated/DS1_valid.csv", index=False, header=False)
pd.DataFrame(testSet).to_csv("dataGenerated/DS1_test.csv", index=False, header=False)